In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
x=np.linspace(-5,5,1000)

In [16]:
fig = make_subplots(
    rows=5, cols=2,
    specs=[[{}, {"rowspan": 2}],
           [{}, None],
           [{"rowspan": 2, "colspan": 2}, None],
           [None, None],
           [{}, {}]],
    print_grid=False)

fig.add_trace(go.Scatter(x=x, y=np.sin(x)**3, name="Sin(x)"), row=1, col=1)
fig.add_trace(go.Scatter(x=x, y=x**2, name="(1,2)"), row=1, col=2)
fig.add_trace(go.Scatter(x=x[500:], y=np.sqrt(x[500:]), name="(2,1)"), row=2, col=1)
fig.add_trace(go.Scatter(x=x, y=10*np.cos(x)**2 + 50*np.sin(x)**3, name="(3,1)"), row=3, col=1)
fig.add_trace(go.Scatter(x=[1, 2], y=[1, 2], name="(5,1)"), row=5, col=1)
fig.add_trace(go.Scatter(x=[1, 2], y=[1, 2], name="(5,2)"), row=5, col=2)

fig.update_layout(height=800, width=1000, title_text="specs examples")
fig.show()

In [19]:
import plotly as py

In [26]:
trace = (go.Scatter(x=x, y= np.sin(x),
                fill='tozeroy', 
#                     fillcolor='green',
 #                    fillcolor='rgb(26,150,65,0.5)',
 #                    fillcolor=dict(color='rgb(26,150,65,0.5)'),
#              fillcolor=dict(marker=dict(color='rgb(26,150,65,0.5)')),
                opacity=0.1,
                showlegend=False,
                hoverinfo='none',
                mode='none'))
py.offline.iplot(go.Figure(data=[trace]))

In [27]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
df.index = df.Date
df = df[['AAPL.Close', 'mavg']]
df['mavg2'] = df['AAPL.Close'].rolling(window=50).mean()
df.columns = ['y', 'ma1', 'ma2']
df=df.tail(250).dropna()
df1 = df.copy()

# split data into chunks where averages cross each other
df['label'] = np.where(df['ma1']>df['ma2'], 1, 0)
df['group'] = df['label'].ne(df['label'].shift()).cumsum()
df = df.groupby('group')
dfs = []
for name, data in df:
    dfs.append(data)

# custom function to set fill color
def fillcol(label):
    if label >= 1:
        return 'rgba(0,250,0,0.4)'
    else:
        return 'rgba(250,0,0,0.4)'

fig = go.Figure()

for df in dfs:
    fig.add_traces(go.Scatter(x=df.index, y = df.ma1,
                              line = dict(color='rgba(0,0,0,0)')))
    
    fig.add_traces(go.Scatter(x=df.index, y = df.ma2,
                              line = dict(color='rgba(0,0,0,0)'),
                              fill='tonexty', 
                              fillcolor = fillcol(df['label'].iloc[0])))

# include averages
fig.add_traces(go.Scatter(x=df1.index, y = df1.ma1,
                          line = dict(color = 'blue', width=1)))

fig.add_traces(go.Scatter(x=df1.index, y = df1.ma2,
                          line = dict(color = 'red', width=1)))

# include main time-series
fig.add_traces(go.Scatter(x=df1.index, y = df1.y,
                          line = dict(color = 'black', width=2)))

fig.update_layout(showlegend=False)
fig.show()